In [1]:
import pandas as pd
import numpy as np
health_df = pd.read_csv("../data_csv/health_timeseries_core_state.csv")

# 컬럼 정규화
health_df = (
    health_df
    .rename(columns={
        "user_id": "asset_id",
        "health_state_index": "state_value"
    })
    .sort_values(["asset_id", "date"])
    .reset_index(drop=True)
)

# t_index 생성
health_df["t_index"] = (
    health_df
    .groupby("asset_id")
    .cumcount()
)

health_df.head()

,asset_id,date,mean_hr,hr_std,steps,calories,sleep_minutes,health_state_level,health_state_speed,state_value,t_index
0,1503960366,2016-04-12,NaN,NaN,13162,1985,327.0,-0.002802,0.000000,-0.002802,0
1,1503960366,2016-04-13,NaN,NaN,10735,1797,384.0,0.054147,0.056949,0.452790,1
2,1503960366,2016-04-14,NaN,NaN,10460,1776,NaN,NaN,NaN,NaN,2
3,1503960366,2016-04-15,NaN,NaN,9762,1745,412.0,0.064168,NaN,NaN,3
4,1503960366,2016-04-16,NaN,NaN,12669,1863,340.0,0.005229,NaN,NaN,4


Core 5에서 쓰던 degradation_rate 재계산 - 예측 구조 유지

In [2]:
health_df["delta_state"] = (
    health_df
    .groupby("asset_id")["state_value"]
    .diff()
)

health_df["degradation_rate"] = (
    health_df
    .groupby("asset_id")["delta_state"]
    .rolling(window=7, min_periods=3)
    .mean()
    .reset_index(level=0, drop=True)
)

health_df[
    ["asset_id", "date", "t_index", "state_value", "degradation_rate"]
].head(10)

,asset_id,date,t_index,state_value,degradation_rate
0,1503960366,2016-04-12,0,-0.002802,NaN
1,1503960366,2016-04-13,1,0.452790,NaN
2,1503960366,2016-04-14,2,NaN,NaN
3,1503960366,2016-04-15,3,NaN,NaN
4,1503960366,2016-04-16,4,NaN,NaN
5,1503960366,2016-04-17,5,NaN,NaN
6,1503960366,2016-04-18,6,NaN,NaN
7,1503960366,2016-04-19,7,NaN,NaN
8,1503960366,2016-04-20,8,NaN,NaN
9,1503960366,2016-04-21,9,NaN,NaN


μHSM 결과 로드 (Core 6 결과)

In [3]:
muhsm_df = pd.read_csv("../data_csv/muHSM_state_monitor.csv")

muhsm_df = (
    muhsm_df
    .rename(columns={"user_id": "asset_id"})
    .sort_values(["asset_id", "date"])
    .reset_index(drop=True)
)

muhsm_df.head()

,asset_id,date,HSI,HDR,recovery_margin,observability_score
0,1503960366,2016-04-12,-0.002802,0.000000,NaN,0.63871
1,1503960366,2016-04-13,0.452790,0.056949,NaN,0.63871
2,1503960366,2016-04-14,NaN,NaN,NaN,0.63871
3,1503960366,2016-04-15,NaN,NaN,NaN,0.63871
4,1503960366,2016-04-16,NaN,NaN,0.2,0.63871


Core 5 로그 + μHSM 상태 merge

In [4]:
core9_df = health_df.merge(
    muhsm_df,
    on=["asset_id", "date"],
    how="left"
)

core9_df[
    ["asset_id", "date", "state_value", "degradation_rate",
     "HSI", "HDR", "recovery_margin", "observability_score"]
].head(10)

,asset_id,date,state_value,degradation_rate,HSI,HDR,recovery_margin,observability_score
0,1503960366,2016-04-12,-0.002802,NaN,-0.002802,0.000000,NaN,0.63871
1,1503960366,2016-04-13,0.452790,NaN,0.452790,0.056949,NaN,0.63871
2,1503960366,2016-04-14,NaN,NaN,NaN,NaN,NaN,0.63871
3,1503960366,2016-04-15,NaN,NaN,NaN,NaN,NaN,0.63871
4,1503960366,2016-04-16,NaN,NaN,NaN,NaN,0.200000,0.63871
5,1503960366,2016-04-17,NaN,NaN,NaN,NaN,0.166667,0.63871
6,1503960366,2016-04-18,NaN,NaN,NaN,NaN,0.142857,0.63871
7,1503960366,2016-04-19,NaN,NaN,NaN,NaN,0.125000,0.63871
8,1503960366,2016-04-20,NaN,NaN,NaN,NaN,0.111111,0.63871
9,1503960366,2016-04-21,NaN,NaN,NaN,NaN,0.100000,0.63871


Core 9 판단 규칙 (μHSM 기반) - 규칙 수 동일 / 임계값 유지

In [5]:
def state_based_decision(row):
    if (
        row["HDR"] < -0.05 and
        row["recovery_margin"] < 0.3 and
        row["observability_score"] > 0.5
    ):
        return 1
    return 0


core9_df["intervention_flag_core9"] = core9_df.apply(
    state_based_decision,
    axis=1
)

core9_df[
    ["asset_id", "date", "HDR", "recovery_margin",
     "observability_score", "intervention_flag_core9"]
].head(10)

,asset_id,date,HDR,recovery_margin,observability_score,intervention_flag_core9
0,1503960366,2016-04-12,0.000000,NaN,0.63871,0
1,1503960366,2016-04-13,0.056949,NaN,0.63871,0
2,1503960366,2016-04-14,NaN,NaN,0.63871,0
3,1503960366,2016-04-15,NaN,NaN,0.63871,0
4,1503960366,2016-04-16,NaN,0.200000,0.63871,0
5,1503960366,2016-04-17,NaN,0.166667,0.63871,0
6,1503960366,2016-04-18,NaN,0.142857,0.63871,0
7,1503960366,2016-04-19,NaN,0.125000,0.63871,0
8,1503960366,2016-04-20,NaN,0.111111,0.63871,0
9,1503960366,2016-04-21,NaN,0.100000,0.63871,0


stabilized 결과 계산 (Core 5와 동일) - 미래 정보 처리 방식 유지

In [6]:
def compute_stabilization(df, window=7):
    df = df.copy()

    df["post_state"] = (
        df
        .groupby("asset_id")["state_value"]
        .shift(-window)
    )

    df["stabilized"] = (
        df["post_state"] - df["state_value"]
    ) > 0

    return df


core9_df = compute_stabilization(core9_df)

Core 5 vs Core 9-B 비교 요약

In [7]:
summary_core9 = (
    core9_df
    .groupby("intervention_flag_core9")
    .agg(
        stabilization_rate=("stabilized", "mean"),
        count=("stabilized", "size")
    )
)

summary_core9

,stabilization_rate,count
intervention_flag_core9,,
0,0.147666,921
1,0.607143,28


Core 9-B 결과 CSV 저장

In [8]:
output_cols = [
    "asset_id",
    "date",
    "t_index",
    "state_value",
    "degradation_rate",
    "HSI",
    "HDR",
    "recovery_margin",
    "observability_score",
    "intervention_flag_core9",
    "stabilized"
]

output_path = "../data_csv/core9_state_based_decision_log.csv"

core9_df[output_cols].to_csv(
    output_path,
    index=False,
    encoding="utf-8-sig"
)

print(f"✅ Core 9-B decision log 저장 완료 → {output_path}")

✅ Core 9-B decision log 저장 완료 → ../data_csv/core9_state_based_decision_log.csv
